# 2D Scanning
## Import libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import henon_map as hm

from parameters import *

## Computing

In [2]:
data_5 = {}
DA_5 = {}
error_5 = {}

alpha_values = np.linspace(0, np.pi * 0.5, alpha_max_samples)

for epsilon in tqdm(epsilons, desc="2D scan"):
    henon_engine = hm.radial_scan.generate_instance(d_r, alpha_values, np.zeros(alpha_values.shape), np.zeros(alpha_values.shape), epsilon)
    radiuses = henon_engine.compute(turn_sampling)
    data_5[(epsilon)] = radiuses

    skips = [1]
    while True:
        if (alpha_max_samples - 1) // skips[-1] > 4:
            skips.append(skips[-1] * 2)
        else:
            break
    
    for i in skips:
        alpha = alpha_values[::i]
        d_alpha = alpha[1] - alpha[0]
        cutted_radiuses = radiuses[::i]
        value = integrate.simps(cutted_radiuses ** 2, alpha, axis=0)
        less_value = integrate.simps(cutted_radiuses[::2] ** 2, alpha[::2], axis=0)
        uncertainty = np.abs((value - less_value))
        
        DA = np.sqrt(value * 2 / np.pi)
        uncertainty = 0.5 * np.power(value * 2 / np.pi, -0.5) * uncertainty
        DA_5[(epsilon, cutted_radiuses.shape)] = np.asarray(DA)
        error_5[(epsilon, cutted_radiuses.shape)] = uncertainty 

2D scan: 100%|██████████| 1/1 [00:05<00:00,  5.42s/it]


### Saving Data

In [3]:
with open("data/DA_5.pkl", 'wb') as f:
    pickle.dump(DA_5, f, protocol=4)
    
with open("data/error_5.pkl", 'wb') as f:
    pickle.dump(error_5, f, protocol=4)